# Creating Ground Truth Dataset

In [73]:
import pandas as pd 
import json
import ast
import re
# Fuzzy Matching with FuzzyWuzzy
from fuzzywuzzy import fuzz
from collections import defaultdict

In [74]:
# Import sample data
sample_df = pd.read_csv('input_data/normalized_eval_samples.csv')
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,address_state_norm,address_postcode_norm,base_address_street_norm,base_address_city_norm,base_address_state_norm,base_address_postcode_norm,address_norm,base_address_norm,categories_norm,base_categories_norm
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': ""Goin' Postal Jacksonville""}","{'primary': 'shipping_center', 'alternate': ['...",0.996261,['http://www.goinpostaljacksonville.com/'],['https://www.facebook.com/463273470392736'],NaN,"[""+19049989600""]",...,FL,32256,7643 gate pkwy ste 104,jacksonville,FL,32256,"{'address': '7643 gate pkwy', 'city': 'jackson...","{'address': '7643 gate pkwy ste 104', 'city': ...","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['http://valleytransca.com/'],['https://www.facebook.com/107150256087765'],NaN,"[""+16194474353""]",...,CA,92021,1158 n 2nd st,el cajon,CA,92021,"{'address': '1158 n 2nd st', 'city': 'el cajon...","{'address': '1158 n 2nd st', 'city': 'el cajon...","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,['https://stores.redwingshoes.com/roswell-ga'],NaN,NaN,"[""7705584400""]",...,GA,30076,875 mansell rd ste b1,roswell,GA,30076,"{'address': '875 mansell rd b1', 'city': 'rosw...","{'address': '875 mansell rd ste b1', 'city': '...","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
3,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,['http://www.newportinstitute.com/?utm_source=...,NaN,NaN,"[""8445450602""]",...,CA,92590,43821 shady creek,temecula,CA,92590,"{'address': '43821 shady crk', 'city': 'temecu...","{'address': '43821 shady creek', 'city': 'teme...","['doctor', 'health_and_medical']","['doctor', 'health_and_medical']"
4,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,['https://www.choicehotels.com/virginia/alexan...,NaN,NaN,"[""7036423422""]",...,VA,22312,6254 duke st,alexandria,VA,22312,"{'address': '6254 duke st', 'city': 'alexandri...","{'address': '6254 duke st', 'city': 'alexandri...",['hotel'],['hotel']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,08f44c10dd244c09032cb0144487c85b,dba914f3e9e247fb64f813c7,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Edward Jones- Financial Advisor:H...,"{'primary': 'financial_advising', 'alternate':...",0.231548,['http://www.edwardjones.com/'],['https://www.facebook.com/536502823203299'],NaN,"[""+16783446501""]",...,GA,30078,2795 main st w ste 24b,snellville,GA,30078,"{'address': '2795 main st w', 'city': 'snellvi...","{'address': '2795 main st w ste 24b', 'city': ...","['financial_advising', 'financial_service']",['business_and_professional_services_>_financi...
1007,08f28d542c8d230903943451a3bc73cf,4bc4c2c3abf49521d686c593,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Starbucks'},"{'primary': 'coffee_shop', 'alternate': ['rest...",0.770000,['https://www.starbucks.com/store-locator/stor...,NaN,NaN,"[""2067552683""]",...,WA,98101,400 pine st,seattle,WA,98101,"{'address': '1400 6th ave', 'city': 'seattle',...","{'address': '400 pine st', 'city': 'seattle', ...","['coffee_shop', 'restaurant']","['dining_and_drinking_>_cafe,_coffee

In [75]:
# Import Yelp dataset
yelp_df = pd.read_csv('input_data/normalized_yelp.csv')
yelp_df.head()

,business_id,name,address,city,state,postal_code,categories,address_norm,city_norm,state_norm,postal_code_norm,name_norm,categories_norm,address_dict_norm
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,"Doctors, Traditional Chinese Medicine, Naturop...","1616 chapala st, ste 2",santa barbara,CA,93101,abby rappoport lac cmq,"['doctors', 'traditional_chinese_medicine', 'n...","{'address': '1616 chapala st, ste 2', 'city': ..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",87 grasso plaza shopping center,affton,MO,63123,ups store,"['shipping_centers', 'local_services', 'notari...","{'address': '87 grasso plaza shopping center',..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,"Department Stores, Shopping, Fashion, Home & G...",5255 e broadway blvd,tucson,AZ,85711,target,"['department_stores', 'shopping', 'fashion', '...","{'address': '5255 e broadway blvd', 'city': 't..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",935 race st,philadelphia,PA,19107,st honore pastries,"['restaurants', 'food', 'bubble_tea', 'coffee_...","{'address': '935 race st', 'city': 'philadelph..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,"Brewpubs, Breweries, Food",101 walnut st,green lane,PA,18054,perkiomen valley brewery,"['brewpubs', 'breweries', 'food']","{'address': '101 walnut st', 'city': 'green la..."


In [76]:
# Parse string columns back to dictionaries and lists

def safe_eval(x):
    """Safely evaluate string representation of dict/list back to Python object"""
    if pd.isna(x) or x == '' or x == 'nan':
        return None
    if isinstance(x, (dict, list)):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return x

sample_df['address_norm'] = sample_df['address_norm'].apply(safe_eval)
sample_df['base_address_norm'] = sample_df['base_address_norm'].apply(safe_eval)

sample_df['categories_norm'] = sample_df['categories_norm'].apply(safe_eval)
sample_df['base_categories_norm'] = sample_df['base_categories_norm'].apply(safe_eval)

yelp_df['address_dict_norm'] = yelp_df['address_dict_norm'].apply(safe_eval)

yelp_df['categories_norm'] = yelp_df['categories_norm'].apply(safe_eval)

## Fuzzy Matching: Sample Data vs Yelp Data

This section uses fuzzy matching to find the best Yelp business for each sample record, then evaluates which attributes (base vs non-base) are most accurate.

In [77]:
def fuzzy_name_similarity(name1, name2):
    """Calculate fuzzy similarity between two names using token sort ratio"""
    if not name1 or not name2:
        return 0.0
    return fuzz.token_sort_ratio(name1, name2) / 100.0

def fuzzy_address_similarity(addr_dict1, addr_dict2):
    """Calculate fuzzy similarity between two address dictionaries"""
    if not addr_dict1 or not addr_dict2:
        return 0.0
    
    # Weight different address components
    weights = {
        'address': 0.4,  # Street address most important
        'city': 0.3,     # City very important
        'state': 0.2,    # State important
        'postal_code': 0.1  # Postal code helpful but less critical
    }
    
    total_similarity = 0.0
    for component, weight in weights.items():
        val1 = addr_dict1.get(component, '')
        val2 = addr_dict2.get(component, '')
        
        if val1 or val2:
            # Use fuzzy matching for addresses
            similarity = fuzz.ratio(val1, val2) / 100.0
            total_similarity += similarity * weight
    
    return total_similarity

def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets"""
    if not set1 and not set2:
        return 1.0
    if not set1 or not set2:
        return 0.0
    
    s1 = set(set1)
    s2 = set(set2)
    
    intersection = len(s1.intersection(s2))
    union = len(s1.union(s2))
    
    return intersection / union if union > 0 else 0.0

def find_best_match(sample_row, yelp_df, max_candidates=1000):
    """Find the best matching Yelp business for a sample row using fuzzy matching"""
    
    # Get normalized attributes from sample row
    sample_name = sample_row['name_norm']
    sample_base_name = sample_row['base_name_norm']
    sample_address = sample_row['address_norm']
    sample_base_address = sample_row['base_address_norm']
    sample_state = sample_row['address_norm'].get('state', '') if sample_row['address_norm'] else ''
    
    # Pre-filter Yelp data by state to reduce search space
    if sample_state:
        yelp_candidates = yelp_df[yelp_df['state_norm'] == sample_state].head(max_candidates)
    else:
        yelp_candidates = yelp_df.head(max_candidates)
    
    if len(yelp_candidates) == 0:
        yelp_candidates = yelp_df.head(max_candidates)
    
    best_match = None
    best_score = 0.0
    
    for idx, yelp_row in yelp_candidates.iterrows():
        yelp_name = yelp_row['name_norm']
        yelp_address = yelp_row['address_dict_norm']
        
        # Calculate name similarities (try both base and non-base)
        name_sim = fuzzy_name_similarity(sample_name, yelp_name)
        base_name_sim = fuzzy_name_similarity(sample_base_name, yelp_name)
        best_name_sim = max(name_sim, base_name_sim)
        
        # Calculate address similarities (try both base and non-base)
        addr_sim = fuzzy_address_similarity(sample_address, yelp_address)
        base_addr_sim = fuzzy_address_similarity(sample_base_address, yelp_address)
        best_addr_sim = max(addr_sim, base_addr_sim)
        
        # Combined score (weighted average)
        combined_score = (best_name_sim * 0.6) + (best_addr_sim * 0.4)
        
        if combined_score > best_score:
            best_score = combined_score
            best_match = {
                'yelp_idx': idx,
                'yelp_business_id': yelp_row['business_id'],
                'yelp_name': yelp_name,
                'yelp_address': yelp_address,
                'yelp_categories': yelp_row.get('categories_norm', []),
                'name_similarity': name_sim,
                'base_name_similarity': base_name_sim,
                'address_similarity': addr_sim,
                'base_address_similarity': base_addr_sim,
                'combined_score': combined_score
            }
    
    return best_match

# Test on one sample

if len(sample_df) > 0:
    test_row = sample_df.iloc[0]
    print(f"Testing on first sample row...")
    print(f"Sample name: '{test_row['name_norm']}'")
    print(f"Sample address: {test_row['address_norm']}")
    
    test_match = find_best_match(test_row, yelp_df)
    if test_match:
        print(f"\nBest match found:")
        print(f"  Yelp name: '{test_match['yelp_name']}'")
        print(f"  Combined score: {test_match['combined_score']:.3f}")
        print(f"  Name similarity: {test_match['name_similarity']:.3f}")
        print(f"  Address similarity: {test_match['address_similarity']:.3f}")
    else:
        print("No match found!")
    

Testing on first sample row...
Sample name: 'goin' postal jacksonville'
Sample address: {'address': '7643 gate pkwy', 'city': 'jacksonville', 'state': 'FL', 'postal_code': '32256'}

Best match found:
  Yelp name: 'pet trailer mobile grooming'
  Combined score: 0.519
  Name similarity: 0.550
  Address similarity: 0.472


In [78]:
# Generate Ground Truth Labels with Fuzzy Matching

def generate_ground_truth():
    """Generate ground truth labels for all sample rows"""
    
    print("=== GENERATING GROUND TRUTH WITH FUZZY MATCHING ===")
    print(f"Processing sample records...")
    
    results = []
    
    for idx, sample_row in sample_df.iterrows():
        # Find best Yelp match
        best_match = find_best_match(sample_row, yelp_df)
        
        if not best_match:
            print(f"Warning: No match found for sample row {idx}")
            continue
        
        # Get attributes for comparison
        sample_name = sample_row['name_norm']
        sample_base_name = sample_row['base_name_norm']
        sample_cats = sample_row.get('categories_norm', [])
        sample_base_cats = sample_row.get('base_categories_norm', [])
        sample_addr = sample_row['address_norm']
        sample_base_addr = sample_row['base_address_norm']
        
        yelp_name = best_match['yelp_name']
        yelp_cats = best_match['yelp_categories']
        yelp_addr = best_match['yelp_address']
        
        # Determine best attributes
        name_sim = best_match['name_similarity']
        base_name_sim = best_match['base_name_similarity']
        
        cat_sim = jaccard_similarity(sample_cats, yelp_cats)
        base_cat_sim = jaccard_similarity(sample_base_cats, yelp_cats)
        
        addr_sim = best_match['address_similarity']
        base_addr_sim = best_match['base_address_similarity']
        
        # Determine winners
        best_name = 'non_base' if name_sim > base_name_sim else ('base' if base_name_sim > name_sim else 'tie')
        best_categories = 'non_base' if cat_sim > base_cat_sim else ('base' if base_cat_sim > cat_sim else 'tie')
        best_address = 'non_base' if addr_sim > base_addr_sim else ('base' if base_addr_sim > addr_sim else 'tie')
        
        # Store results
        result = {
            'sample_idx': idx,
            'sample_id': sample_row.get('id', idx),
            'yelp_idx': best_match['yelp_idx'],
            'yelp_business_id': best_match['yelp_business_id'],
            'match_confidence': best_match['combined_score'],
            
            # Name comparison
            'best_name_source': best_name,
            'name_similarity': name_sim,
            'base_name_similarity': base_name_sim,
            'sample_name': sample_name,
            'sample_base_name': sample_base_name,
            'yelp_name': yelp_name,
            
            # Categories comparison  
            'best_categories_source': best_categories,
            'categories_similarity': cat_sim,
            'base_categories_similarity': base_cat_sim,
            'sample_categories': sample_cats,
            'sample_base_categories': sample_base_cats,
            'yelp_categories': yelp_cats,
            
            # Address comparison
            'best_address_source': best_address,
            'address_similarity': addr_sim,
            'base_address_similarity': base_addr_sim,
            'sample_address': sample_addr,
            'sample_base_address': sample_base_addr,
            'yelp_address': yelp_addr
        }
        
        results.append(result)
    
    return results

# Generate the ground truth labels
ground_truth_results = generate_ground_truth()

# Convert to DataFrame for easier analysis
ground_truth_df = pd.DataFrame(ground_truth_results)

=== GENERATING GROUND TRUTH WITH FUZZY MATCHING ===
Processing sample records...


In [79]:
# Analyze Ground Truth Coverage
print("=== GROUND TRUTH COVERAGE ANALYSIS ===")

if len(ground_truth_df) > 0:
    total_records = len(ground_truth_df)
    print(f"Total matched pairs: {total_records}\n")
    
    # Analyze coverage for core attributes
    print("NON-NULL COVERAGE ACROSS CORE ATTRIBUTES:")
    print("=" * 60)
    
    # Names coverage
    print("\nNAMES:")
    name_coverage = (ground_truth_df['sample_name'].notna() & (ground_truth_df['sample_name'] != '')).sum()
    base_name_coverage = (ground_truth_df['sample_base_name'].notna() & (ground_truth_df['sample_base_name'] != '')).sum()
    yelp_name_coverage = (ground_truth_df['yelp_name'].notna() & (ground_truth_df['yelp_name'] != '')).sum()
    
    print(f"  Sample names:      {name_coverage:4d} / {total_records} ({name_coverage/total_records*100:5.1f}%)")
    print(f"  Base names:        {base_name_coverage:4d} / {total_records} ({base_name_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp names:        {yelp_name_coverage:4d} / {total_records} ({yelp_name_coverage/total_records*100:5.1f}%)")
    
    # Addresses coverage
    print("\nADDRESSES:")
    addr_coverage = ground_truth_df['sample_address'].apply(lambda x: x is not None and isinstance(x, dict) and any(v for v in x.values() if v)).sum()
    base_addr_coverage = ground_truth_df['sample_base_address'].apply(lambda x: x is not None and isinstance(x, dict) and any(v for v in x.values() if v)).sum()
    yelp_addr_coverage = ground_truth_df['yelp_address'].apply(lambda x: x is not None and isinstance(x, dict) and any(v for v in x.values() if v)).sum()
    
    print(f"  Sample addresses:  {addr_coverage:4d} / {total_records} ({addr_coverage/total_records*100:5.1f}%)")
    print(f"  Base addresses:    {base_addr_coverage:4d} / {total_records} ({base_addr_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp addresses:    {yelp_addr_coverage:4d} / {total_records} ({yelp_addr_coverage/total_records*100:5.1f}%)")
    
    # Categories coverage
    print("\nCATEGORIES:")
    cat_coverage = ground_truth_df['sample_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    base_cat_coverage = ground_truth_df['sample_base_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    yelp_cat_coverage = ground_truth_df['yelp_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    
    print(f"  Sample categories: {cat_coverage:4d} / {total_records} ({cat_coverage/total_records*100:5.1f}%)")
    print(f"  Base categories:   {base_cat_coverage:4d} / {total_records} ({base_cat_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp categories:   {yelp_cat_coverage:4d} / {total_records} ({yelp_cat_coverage/total_records*100:5.1f}%)")
    
    # Summary statistics
    print("\n" + "=" * 60)
    print("MATCH QUALITY STATISTICS:")
    print("=" * 60)
    print(f"Average match confidence: {ground_truth_df['match_confidence'].mean():.3f}")
    print(f"Match confidence std:     {ground_truth_df['match_confidence'].std():.3f}")
    print(f"Min confidence:           {ground_truth_df['match_confidence'].min():.3f}")
    print(f"Max confidence:           {ground_truth_df['match_confidence'].max():.3f}")
    
    # Attribute winner distribution
    print("\n" + "=" * 60)
    print("BEST ATTRIBUTE SOURCE DISTRIBUTION:")
    print("=" * 60)
    
    print("\nNames:")
    name_counts = ground_truth_df['best_name_source'].value_counts()
    for source, count in name_counts.items():
        pct = (count / total_records) * 100
        print(f"  {source:10s}: {count:4d} ({pct:5.1f}%)")
    
    print("\nCategories:")
    cat_counts = ground_truth_df['best_categories_source'].value_counts()
    for source, count in cat_counts.items():
        pct = (count / total_records) * 100
        print(f"  {source:10s}: {count:4d} ({pct:5.1f}%)")
    
    print("\nAddresses:")
    addr_counts = ground_truth_df['best_address_source'].value_counts()
    for source, count in addr_counts.items():
        pct = (count / total_records) * 100
        print(f"  {source:10s}: {count:4d} ({pct:5.1f}%)")
    
else:
    print("No ground truth results to analyze!")

# Export
ground_truth_df.to_csv('labeled_ground_truth.csv', index=False)

=== GROUND TRUTH COVERAGE ANALYSIS ===
Total matched pairs: 1011

NON-NULL COVERAGE ACROSS CORE ATTRIBUTES:

NAMES:
  Sample names:      1011 / 1011 (100.0%)
  Base names:        1011 / 1011 (100.0%)
  Yelp names:        1011 / 1011 (100.0%)

ADDRESSES:
  Sample addresses:  1011 / 1011 (100.0%)
  Base addresses:    1011 / 1011 (100.0%)
  Yelp addresses:    1011 / 1011 (100.0%)

CATEGORIES:
  Sample categories: 1000 / 1011 ( 98.9%)
  Base categories:    986 / 1011 ( 97.5%)
  Yelp categories:   1011 / 1011 (100.0%)

MATCH QUALITY STATISTICS:
Average match confidence: 0.496
Match confidence std:     0.123
Min confidence:           0.157
Max confidence:           1.000

BEST ATTRIBUTE SOURCE DISTRIBUTION:

Names:
  tie       :  661 ( 65.4%)
  non_base  :  182 ( 18.0%)
  base      :  168 ( 16.6%)

Categories:
  tie       :  953 ( 94.3%)
  non_base  :   31 (  3.1%)
  base      :   27 (  2.7%)

Addresses:
  tie       :  596 ( 59.0%)
  non_base  :  230 ( 22.7%)
  base      :  185 ( 18.3%)
